In [1]:
import os
import sys

sys.path.append(os.pardir)

In [2]:
from PIL import Image
from IPython.display import display
from dataset.mnist import load_mnist

import pickle
import numpy as np

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def softmax(x):
    c = np.max(x)
    
    exp_x = np.exp(x-c) # 오버플로우 처리
    
    sum_exp_x = np.sum(exp_x)
    
    y = exp_x / sum_exp_x
    
    return y

In [5]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) # NumPy 배열을 PIL 데이터 객체로 변환
    display(pil_img)

In [6]:
(X_train, y_train), (X_test, y_test) = load_mnist(flatten=True, normalize=False)

In [7]:
img = X_train[6]
label = y_train[6]

print(label)

1


In [8]:
img_show(img.reshape(28, 28))

In [9]:
def get_data():
    (X_train, y_train), (X_test, y_test) = load_mnist(flatten=True, normalize=False, one_hot_label=False)
    
    return X_test, y_test

def init_network():
    with open('sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1)+b1
    z1 = sigmoid(a1)
    
    a2 = np.dot(z1, W2)+b2
    z2 = sigmoid(a2)
    
    a3 = np.dot(z2, W3)+b3
    
    y = softmax(a3)
    
    return y

In [10]:
X, y = get_data()

network = init_network()

accuracy_cnt = 0

for i in range(len(X)):
    pred = predict(network, X[i])
    p = np.argmax(pred)
    
    if p == y[i]:
        accuracy_cnt += 1
    
print(f'Accuracy: {accuracy_cnt/len(X)}')

C:\Users\epdls\AppData\Local\Temp\ipykernel_12324\4033946986.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Accuracy: 0.9207


In [11]:
for key, item in network.items():
    print(f'{key} : {item.shape}')

b2 : (100,)
W1 : (784, 50)
b1 : (50,)
W2 : (50, 100)
W3 : (100, 10)
b3 : (10,)


## Batch 처리

In [12]:
X, y = get_data()

network = init_network()

batch_size, accuracy_cnt = 100, 0

for i in range(0, len(X), batch_size):
    X_batch = X[i:i+batch_size]
    y_batch = predict(network, X_batch)
    
    p = np.argmax(y_batch, axis=1)
    
    accuracy_cnt += np.sum(p==y[i:i+batch_size]) # np.sum(~): 비교 후 True 개수 카운트
    
print(f'Accuracy: {accuracy_cnt/len(X)}')

Accuracy: 0.9207


C:\Users\epdls\AppData\Local\Temp\ipykernel_12324\4033946986.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
